This blog post describes a possible use case of the `fumbbl_replays` package.

To install the `fumbbl_replays` python package, follow the instructions on Github.

# Application: Plotting defensive tournament setups

Here the idea is to visualize defensive setups in a tournament setting.

To test the idea, I chose the Tilean Team Cup. It was an online NAF tournament in 2023 on FUMBBL that used the World Cup ruleset.
https://member.thenaf.net/index.php?module=NAF&type=tournaments&func=view&id=7495

NAF tournament director Stimme wrote:

https://www.thenaf.net/2023/05/tournament-director-blog-may-2023/

*Among the individual coaches, Siggi stood out with his Amazons, earning the title of best coach with a flawless record of six wins. Kurjo’s Orcs secured second place, while helborg’s Dark Elves came in third, both with five wins and a draw.* 


In [ ]:
import pandas as pd
import fumbbl_replays as fb

# point this to the location of the CSV datasets
path_to_datasets = '../fumbbl_scraping/datasets/current/'

# FUMBBL matches
target = 'df_matches.csv'
df_matches = pd.read_csv(path_to_datasets + target) 

# # subset on tilean team cup
df_matches = df_matches.query('tournament_id == 59383')

sel1 = df_matches['coach1_ranking'] == 'siggi'
sel2 = df_matches['coach2_ranking'] == 'siggi'

df_matches = df_matches[(sel1 + sel2)]

tilean_replays = df_matches['match_id'].values

tilean_replays = tilean_replays[0:4]

In [ ]:
tilean_replays

In [ ]:
import os

target_dir = 'kickoff_pngs'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

fullrun = 1

if fullrun:
    id = []
    match_ids = []
    race_defense = []
    race_offense = []

    for match_id in tilean_replays:
        match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id) 
        plot = fb.write_plot(match_id, positions, receiving_team, metadata, refresh = True, verbose = False)
        id.append(int(replay_id))
        match_ids.append(int(match_id))
        race_defense.append(metadata[4])
        race_offense.append(metadata[5])

    df_replays = pd.DataFrame( {"matchId": match_ids,
                                "replayId": id,
                                "raceOffense": race_offense,
                                "raceDefense": race_defense})
    target = 'kickoff_pngs/df_replays'
    df_replays.to_csv(target + '.csv', index = False)
else:
    # FUMBBL matches
    target = 'kickoff_pngs/df_replays.csv'
    df_replays = pd.read_csv(target)  

In [ ]:
from PIL import Image
Image.open("kickoff_pngs/amazon/1616794_4456080_kickoff_lower_defense.png")

Apparently, coach siggi used an asymmetric setup against his Skaven opponent, with the weakest players on the line as cannon fodder, and positional players with good defensive skills such as block and wrestle further back. The dirty player lineman, Leader Thrower and Guard blocker are safe on row four.

There is also a function that extracts the roster from a replay (N.b. rerolls, apo and inducements not yet implemented):

In [ ]:
my_replay = fb.fetch_replay(match_id = 4447483)
rosters = fb.extract_rosters_from_replay(my_replay)

(rosters
 .query('race == "Amazon"')
 .filter(['short_name', 'positionName', 'playerName', 'skillArrayRoster', 'learned_skills', 'cost', 'recoveringInjury'])
)